In [4]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import json
import numpy as np
import keras.backend as K
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional


Using TensorFlow backend.


In [5]:

# 1. Loading the data
print("loading data...")

pos_file_name = "C:\\Users\\skelenja\\Desktop\\Browserhoistory\\Sentiment-Analysis-for-product-reviews-master\\pos_amazon_cell_phone_reviews.json"
neg_file_name = "C:\\Users\\skelenja\\Desktop\\Browserhoistory\\Sentiment-Analysis-for-product-reviews-master\\neg_amazon_cell_phone_reviews.json"
pos_file = open(pos_file_name, "r")
neg_file = open(neg_file_name, "r")
pos_data = json.loads(pos_file.read())['root']
neg_data = json.loads(neg_file.read())['root']
print("Posititve data loaded. ", len(pos_data), "entries")
print("Negative data loaded. ", len(neg_data), "entries")

print("done loading data...")

plabels = []
nlabels = []

loading data...
Posititve data loaded.  108664 entries
Negative data loaded.  13279 entries
done loading data...


In [6]:

# 2.Process reviews into sentences
pos_sentences, neg_sentences = [], [] 
for entry in pos_data :
    pos_sentences.append(entry['summary'] + " . " + entry['text'])
    plabels.append(1)
for entry in neg_data :
    nlabels.append(0)
    neg_sentences.append(entry['summary'] + " . " + entry['text'])
print(len(pos_sentences))
print(len(neg_sentences))

texts = pos_sentences + neg_sentences 
labels = [1]*len(pos_sentences) + [0]*len(neg_sentences)


108664
13279


In [16]:
labels

array([1, 1, 1, ..., 1, 1, 1])

In [7]:

# 3. Tokenize
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

MAX_SEQUENCE_LENGTH = 50

data = sequence.pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

Found 69671 unique tokens.


In [8]:

# print(labels)

labels = np.array(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (121943, 50)
Shape of label tensor: (121943,)


In [9]:

# split the data into a training set and a validation set

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

rest_data = data[3000:]
rest_labels = labels[3000:]


data = data[:5000]
labels = labels[:5000]

VALIDATION_SPLIT = 0.2
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

print(data.shape, labels.shape, nb_validation_samples)

print(labels)

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print(len(x_train), len(y_train))

(5000, 50) (5000,) 1000
[1 1 1 ..., 1 1 1]
4000 4000


In [11]:

#GloVe
embeddings_index = {}
f = open('C:\\Users\\skelenja\\Desktop\\Browserhoistory\\Sentiment-Analysis-for-product-reviews-master\\glove.6B.50d.txt', 'r', encoding = 'utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

EMBEDDING_DIM = MAX_SEQUENCE_LENGTH

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

def precision(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1))) 
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1))) 
    precision = true_positives / (predicted_positives + K.epsilon()) 
    return precision

def recall(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1))) 
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1))) 
    recall = true_positives / (possible_positives + K.epsilon()) 
    return recall 

Found 400000 word vectors.


In [12]:

# Training the LSTM model

batch_size = 128

model = Sequential()

model.add(embedding_layer)

model.add(LSTM(64))

model.add(Dropout(0.50))

model.add(Dense(1, activation='sigmoid'))

In [13]:
# try using different optimizers and different optimizer configs

model.compile('adam', 'binary_crossentropy', metrics=['accuracy', precision, recall])

print('Train...')

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=16,
          validation_data=[x_val, y_val])

x = model.evaluate(rest_data[:5000], rest_labels[:5000])
print("Loss: ", x[0])
print("Accuracy: ", x[1])
print("Precision: ", x[2])
print("Recall: ", x[3])

Train...
Train on 4000 samples, validate on 1000 samples
Epoch 1/16
4000/4000 [==============================] - 2s 465us/step - loss: 0.3973 - acc: 0.8755 - precision: 0.8925 - recall: 0.9784 - val_loss: 0.3309 - val_acc: 0.8960 - val_precision: 0.8969 - val_recall: 0.9989
Epoch 2/16
4000/4000 [==============================] - 1s 276us/step - loss: 0.3411 - acc: 0.8937 - precision: 0.8937 - recall: 1.0000 - val_loss: 0.3274 - val_acc: 0.8960 - val_precision: 0.8969 - val_recall: 0.9989
Epoch 3/16
4000/4000 [==============================] - 1s 283us/step - loss: 0.3348 - acc: 0.8937 - precision: 0.8937 - recall: 1.0000 - val_loss: 0.3177 - val_acc: 0.8960 - val_precision: 0.8969 - val_recall: 0.9989
Epoch 4/16
4000/4000 [==============================] - 1s 282us/step - loss: 0.3036 - acc: 0.8937 - precision: 0.8940 - recall: 0.9997 - val_loss: 0.2720 - val_acc: 0.8960 - val_precision: 0.8969 - val_recall: 0.9989
Epoch 5/16
4000/4000 [==============================] - 1s 270us/step -